In [2]:
import requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import html5lib
from datetime import datetime as dt
import json
import re

In [2]:
def parseTalosVuln(item):
    text=item.replace("  ","")
    tlist=text.split("  ")[0].split("\n\n")
    finlist={}
    j=1
    taglist=['vuln_name','discovered_on','cve','cvss']
    for i in range(len(tlist)):
        if tlist[i]=="":
            j+=1
            continue
        finlist[taglist[i-j]]=tlist[i].strip()
    return finlist

In [3]:
def talosintelligence(dlimit=30):
    URL = "https://talosintelligence.com/vulnerability_reports"
    req= Request(URL, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'})
    webpage=urlopen(req).read()
    soup=BeautifulSoup(webpage,'html.parser')
    #print(soup.prettify())
    items=[]
    x=[]
    for tag in soup.find_all('tr'):
        x.append(tag)
        items.append(tag.text)
    count=2
    intel_list=[]
    while True: 
        item=items[count]
        idict=parseTalosVuln(item)
        y=str(dt.now()).split(' ')[0]
        diffdays=(dt.strptime(y, "%Y-%m-%d") - dt.strptime(idict['discovered_on'], "%Y-%m-%d")).days
        if diffdays>dlimit:
            break
        count+=1
        intel_list.append(idict)
    return intel_list
#talos_intel=talosintelligence(90)
#print(talos_intel)

In [4]:
def fortinet():
    filepath='logs/fortinet.json'
    URL = "https://www.fortinet.com/fortiguard/outbreak-alert"
    req= Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
    webpage=urlopen(req).read()
    page_soup=BeautifulSoup(webpage,'html.parser')
    items=[]
    cards=page_soup.find_all('div',{'class':'alert-cell'})
    for card in cards:
        title=card.find_all('div',{'class':'alert-title'})[0].text.strip()
        severity=card.find_all('div',{'class':'alert-data'})[0].text.strip().split('\n')[1].split(':')[1].strip()
        adesc=card.find_all('div',{'class':'alert-desc'})
        tags=adesc[0].text.split(':')[1].strip()
        desc=adesc[1].text.split(':')[1].strip()
        link=card.find_all('div',{'class':'alert-link'})[0].find_all('a',href=True)[0]['href'].strip()
        items.append({'source':'FortiGuard Labs Fortinet','title':title,'severity':severity,'tags':tags,'desc:':desc,'link':link})
    #print(items)
    if items != None:
        json.dump(items,open(filepath,'w'))
        return items
    else: 
        return None
#news=fortinet()
#print(news)

In [5]:
def proofPoint():
    filepath='logs/proofpoint.json'
    URL = "https://www.proofpoint.com/us/blog/threat-insight"
    req= Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
    webpage=urlopen(req).read()
    soup=BeautifulSoup(webpage,'html.parser')
    #print(soup.prettify())
    items=[]
    links=soup.find_all("a", {"class": "blog-mosaic__link"})
    titles=soup.find_all("div", {"class": "blog-mosaic__title"})
    #print(titles)
    for i in range(len(links)):
        link=links[i]['href'].strip()
        title=titles[i].text.strip()
        if link!="":
            link="https://www.proofpoint.com"+link
            items.append({"source":"ProofPoint","link":link,"title":title})
    title_main=soup.find_all("a", {"class": "blog-teaser__title"})
    for i in range(len(title_main)):
        link=title_main[i]['href'].strip()
        if link!="":
            link="https://www.proofpoint.com"+link
        title_name=title_main[0].text.strip()
        items.append({"source":"ProofPoint","link":link,"title":title_name})
    #print(items)
    if items != None:
        json.dump(items,open(filepath,'w'))
        return items
    else: 
        return None
#path=proofPoint()
#print(path)

In [6]:
def securityWeek():
    filepath='logs/securityweek.json'
    URL = "https://www.securityweek.com/category/data-breaches/"
    req= Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
    webpage=urlopen(req).read()
    page_soup=BeautifulSoup(webpage,'html.parser')
    #print(page_soup.prettify())
    items=[]
    for tag in page_soup.find_all("div",{"class":"zox-art-title"}):
        link=tag.find_all("a",href=True)[0]['href'].strip()
        items.append({"source":"Security Week","link":link,"title":tag.text.strip()})
    #print(items)
    if items != None:
        json.dump(items,open(filepath,'w'))
        return items
    else: 
        return None
#path=securityWeek()
#print(path)

In [7]:
def getOwaspTop10():
    filepath='logs/owasp.json'
    URL = "https://owasp.org/www-project-top-ten/"
    req= Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
    webpage=urlopen(req).read()
    page_soup=BeautifulSoup(webpage,'html.parser')
    #print(page_soup.prettify())
    items=[]
    sec=page_soup.find_all("section",{"id":"sec-main"})[0]
    links=sec.find_all("a")
    strongs=sec.find_all("strong")
    descs=sec.find_all("li")
    for i in range(len(links)):
        link=links[i]['href'].strip()
        strong=strongs[i].text.strip()
        desc=descs[i].text.strip()
        owaspid=strong.split("-")[0].strip()
        items.append({"_id":owaspid,"link":link,"attack":strong,"desc":desc})
    #print(items)
    if items != None:
        json.dump(items,open(filepath,'w'))
        return items
    else: 
        return None
#path=getOwaspTop10()
#print(path)

In [8]:
def sentinelOne():
    filepath='logs/sentinelone.json'
    URL = "https://www.sentinelone.com/blog/category/from-the-front-lines/"
    req= Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
    webpage=urlopen(req).read()
    page_soup=BeautifulSoup(webpage,'html.parser')
    #print(page_soup.prettify())
    items=[]
    feat=page_soup.find_all("div",{"class":"featured"})
    feat_url=feat[0].find_all("a", href=True)[0]['href'].strip()
    feat_title=feat[0].find_all("h2")[0].text.strip()
    items.append({"title": feat_title,"link": feat_url})
    art=page_soup.find_all("article")
 
    for article in art:
        article_url=article.find_all("a", href=True)[0]['href'].strip()
        article_title=article.find_all("h2")[0].text.strip()
        items.append({"source":"SentinelOne","title": article_title, "link": article_url})
    #print(items)
    if items != None:
        json.dump(items,open(filepath,'w'))
        return items
    else: 
        return None
#path=sentinelOne()
#print(path)

In [9]:
def bleepingComputer():
    filepath='logs/bleepingcomputer.json'
    URL = "https://www.bleepingcomputer.com/tag/data-breach/"
    req= Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
    webpage=urlopen(req).read()
    page_soup=BeautifulSoup(webpage,'html.parser')
    #print(page_soup.prettify())
    news_list=page_soup.find_all("div",{"class":"bc_latest_news_text"})
    items=[]
    for news in news_list:
        extract=news.find_all("h4")
        link=extract[0].find_all('a',href=True)[0]['href'].strip()
        title=extract[0].text.strip()
        desc=news.find_all("p")[0].text.strip()
        items.append({"source":"Bleeping Computer","title":title,"link":link,"desc":desc})
    #print(items)
    if items != None:
        json.dump(items,open(filepath,'w'))
        return items
    else: 
        return None
#path=bleepingComputer()
#print(path)

In [10]:
def tenable():
    filepath='logs/tenable.json'
    URL = "https://www.tenable.com/blog/search?field_blog_section_tid=47"
    req= Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
    webpage=urlopen(req).read()
    page_soup=BeautifulSoup(webpage,'html.parser')
    #print(page_soup.prettify())
    cards=page_soup.find_all('div',{'class':'blog-item__content'})
    items=[]
    for card in cards:
        news=card.find_all('a',href=True)[0]
        link="https://www.tenable.com"+news['href'].strip()
        title=news.text.strip()
        items.append({'source':'Tenable','title':title,'link':link})
    #print(items)
    if items != None:
        json.dump(items,open(filepath,'w'))
        return items
    else: 
        return None
#path=tenable()
#print(path)

In [12]:
def reSecurity():
    filepath='logs/resecurity.json'
    URL = "https://www.resecurity.com/blog"
    req= Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
    webpage=urlopen(req).read()
    page_soup=BeautifulSoup(webpage,'html.parser')
    cards=page_soup.find_all("a",{"class":"col-lg-11"})
    items=[]
    #print(cards)
    for card in cards:
        link="https://www.resecurity.com"+card['href'].strip()
        #print(link)
        text=card.find_all('div',{'class':'text-h4-size news-title'})[0].text.strip()
        #print(text)
        items.append({"source":"ReSecurity","title":text,"link":link})
    #print(items)
    if items != None:
        json.dump(items,open(filepath,'w'))
        return items
    else: 
        return None
items=reSecurity()
#print(items)

In [12]:
def cveSearch(cvename):
    URL = "https://nvd.nist.gov/vuln/detail/"+cvename
    req= Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
    webpage=urlopen(req).read()
    page_soup=BeautifulSoup(webpage,'html.parser')
    #print(page_soup.prettify())
    vuln_desc=page_soup.find_all('p',{'data-testid':'vuln-description'})[0].text.strip()
    print(vuln_desc)
    cvss_ver=page_soup.find_all('div',{'data-testid':'vuln-cvss3-panel'})[0].find_all('strong')[0].text.strip()[:-1]
    sev_vec=page_soup.find_all('span',{'class':'tooltipCvss3CnaMetrics'})[0].text.strip()[9:]
    vec_temp=sev_vec.split("/")
    vectors=[]
    met={'AV':'Attack Vector','AC':'Attack Complexity','PR':'Privileges Required','UI':'User Interaction','S':'Scope','C': 'Confidentiality','I':'Integrity','A':'Availability'}
    val={'AV':{'L':'Local','A':'Adjacent Network','N':'Network','P':'Physical'},'AC':{'H':'High','M':'Medium','L':'Low'},'AU':{'M':'Multiple','S':'Single','N':'None'},'C':{'N':'None','P':'Partial','C':'Complete'},'I':{'N':'None','P':'Partial','C':'Complete'}, 'A':{'N':'None','P':'Partial','C':'Complete'},'UI':{'N':'None','R':'Required'},'PR':{'N':'None','L':'Low','H':'High'},'S':{'U':'Unchanged','C':'Changed'}}
    for vec in vec_temp:
        vc=vec.strip().split(":")
        V=vc[0].strip()
        N=vc[1].strip()
        if V in list(met.keys()) and V in list(val.keys()):
            if N in list(val[V].keys()):
                vectors.append({met[V] : val[V][N]})
    print(vectors)
    
    vuln_hyp_table=page_soup.find_all('table',{'data-testid':'vuln-hyperlinks-table'})[0].find_all('td')
    resources=[]
    links=[]
    res_val=""
    for i in range(len(vuln_hyp_table)):
        if i%2==0:
            link=vuln_hyp_table[i].find_all('a',href=True)[0]['href'].strip()
            links.append(link)
        else:
            res=vuln_hyp_table[i].find_all('span',{'class':'badge'})
            for item in res:
                res_val=res_val+item.text.strip()+", "
            res_val=res_val[:-2]
            resources.append(res_val)
            res_val=""
    print(links,resources)
cveSearch("CVE-2022-40684")
#print(news)

An authentication bypass using an alternate path or channel [CWE-288] in Fortinet FortiOS version 7.2.0 through 7.2.1 and 7.0.0 through 7.0.6, FortiProxy version 7.2.0 and version 7.0.0 through 7.0.6 and FortiSwitchManager version 7.2.0 and 7.0.0 allows an unauthenticated atttacker to perform operations on the administrative interface via specially crafted HTTP or HTTPS requests.
[{'Attack Vector': 'Network'}, {'Attack Complexity': 'Low'}, {'Privileges Required': 'None'}, {'User Interaction': 'None'}, {'Scope': 'Unchanged'}]
['http://packetstormsecurity.com/files/169431/Fortinet-FortiOS-FortiProxy-FortiSwitchManager-Authentication-Bypass.html', 'http://packetstormsecurity.com/files/171515/Fortinet-7.2.1-Authentication-Bypass.html', 'https://fortiguard.com/psirt/FG-IR-22-377'] ['Exploit, Third Party Advisory', '', 'Mitigation, Vendor Advisory']


In [2]:
URL = "https://nvd.nist.gov/vuln/detail/cve-2022-41328"
req= Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
webpage=urlopen(req).read()
page_soup=BeautifulSoup(webpage,'html.parser')
#print(page_soup.prettify())
#cveSearch("CVE-2022-40684")

In [4]:
cvename="cve-2022-41328"
vuln_desc=page_soup.find_all('p',{'data-testid':'vuln-description'})[0].text.strip()
#print(vuln_desc)
#cvss_ver=page_soup.find_all('div',{'data-testid':'vuln-cvss3-panel'})[0].find_all('strong')[0].text.strip()[:-1]
cvss_val=page_soup.find_all('a',{'data-testid':'vuln-cvss3-cna-panel-score'})[0].text.strip().split(" ")
cvss_score=cvss_val[0].strip()
cvss_sev=cvss_val[1].strip()
#print(cvss_score)
sev_vec=page_soup.find_all('span',{'class':'tooltipCvss3CnaMetrics'})[0].text.strip()[9:]
cvss_ver=page_soup.find_all('span',{'class':'tooltipCvss3CnaMetrics'})[0].text.strip()[:8]
vec_temp=sev_vec.split("/")
vectors=[]
met={'AV':'Attack Vector','AC':'Attack Complexity','PR':'Privileges Required','UI':'User Interaction','S':'Scope','C': 'Confidentiality','I':'Integrity','A':'Availability'}
val={'AV':{'L':'Local','A':'Adjacent Network','N':'Network','P':'Physical'},'AC':{'H':'High','M':'Medium','L':'Low'},'AU':{'M':'Multiple','S':'Single','N':'None'},'C':{'N':'None','P':'Partial','C':'Complete'},'I':{'N':'None','P':'Partial','C':'Complete'}, 'A':{'N':'None','P':'Partial','C':'Complete'},'UI':{'N':'None','R':'Required'},'PR':{'N':'None','L':'Low','H':'High'},'S':{'U':'Unchanged','C':'Changed'}}
for vec in vec_temp:
    vc=vec.strip().split(":")
    V=vc[0].strip()
    N=vc[1].strip()
    if V in list(met.keys()) and V in list(val.keys()):
        if N in list(val[V].keys()):
            vectors.append({met[V] : val[V][N]})
#print(vectors)

vuln_hyp_table=page_soup.find_all('table',{'data-testid':'vuln-hyperlinks-table'})[0].find_all('td')
resources=[]
links=[]
res_val=""
for i in range(len(vuln_hyp_table)):
    if i%2==0:
        link=vuln_hyp_table[i].find_all('a',href=True)[0]['href'].strip()
        links.append(link)
    else:
        res=vuln_hyp_table[i].find_all('span',{'class':'badge'})
        for item in res:
            res_val=res_val+item.text.strip()+", "
        res_val=res_val[:-2]
        resources.append(res_val)
        res_val=""
#print(links,resources)

hyp=[]
for i in range(len(links)):
    hyp.append({'link':links[i],'res':resources[i]})
vuln_pub=page_soup.find('span',{'data-testid':'vuln-published-on'}).text.strip()
vuln_last=page_soup.find('span',{'data-testid':'vuln-last-modified-on'}).text.strip()
vuln_src=page_soup.find('span',{'data-testid':'vuln-current-description-source'}).text.strip()
#print(vuln_pub,vuln_last,vuln_src)
vuln_info={'cve':cvename,'version':cvss_ver,'description':vuln_desc, 'base_score':cvss_score,'severity':cvss_sev,'vectors':vectors,'hyperlinks':hyp,'published':vuln_pub,'modified':vuln_last,'source':vuln_src}
print(vuln_info)

{'cve': 'cve-2022-41328', 'version': 'CVSS:3.1', 'description': "A improper limitation of a pathname to a restricted directory vulnerability ('path traversal') [CWE-22] in Fortinet FortiOS version 7.2.0 through 7.2.3, 7.0.0 through 7.0.9 and before 6.4.11 allows a privileged attacker to read and write files on the underlying Linux system via crafted CLI commands.", 'base_score': '6.7', 'severity': 'MEDIUM', 'vectors': [{'Attack Vector': 'Local'}, {'Attack Complexity': 'Low'}, {'Privileges Required': 'High'}, {'User Interaction': 'None'}, {'Scope': 'Unchanged'}], 'hyperlinks': [{'link': 'https://fortiguard.com/psirt/FG-IR-22-369', 'res': 'Vendor Advisory'}], 'published': '03/07/2023', 'modified': '03/14/2023', 'source': 'Fortinet, Inc.'}


In [54]:
def thehackerNews():
    filepath='logs/thehackernews'
    URL = "https://thehackernews.com/search/label/Vulnerability"
    req= Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
    webpage=urlopen(req).read()
    page_soup=BeautifulSoup(webpage,'html.parser')
    cards=page_soup.find_all('div',{'class':'body-post clear'})
    v_items=[]
    for card in cards:
        link=card.find_all('a',href=True)[0]['href'].strip()
        title=card.find_all('h2',{'class':'home-title'})[0].text.strip().replace(u'\xa0', u' ')
        desc=card.find_all('div',{'class':'home-desc'})[0].text.strip().replace(u'\xa0', u' ')
        desc_text=str(desc)
        desc=desc_text.replace("\'","'")
        desc=desc+"..."
        print(desc)
        
        '''
        tags=card.find_all('span',{'class':'h-tags'})
        if len(tags)>=1:
            tags=tags[0].text
        else:
            tags="Vulnerability"
        
        v_items.append({'source':'TheHackerNews','link':link,'tags':tags,'title':title,'desc':desc})
        '''
        v_items.append({'source':'TheHackerNews','link':link,'title':title,'desc':desc})
    
    trending_cards=page_soup.find_all('div',{'class':'clear section'})[0]
    t_title=trending_cards.find_all('div',{'class':'pop-title'})
    t_link=trending_cards.find_all('a',href=True)
    t_items=[]
    for i in range(len(t_link)):
        title=t_title[i].text.strip()
        link=t_link[i]['href'].strip()
        t_items.append({'source':'TheHackerNews','title':title,'link':link})
        
    URL2 = "https://thehackernews.com/search/label/data%20breach"
    req2= Request(URL2, headers={'User-Agent': 'Mozilla/5.0'})
    webpage2=urlopen(req2).read()
    page_soup2=BeautifulSoup(webpage2,'html.parser')
    cards2=page_soup2.find_all('div',{'class':'body-post clear'})
    b_items=[]
    for card in cards2:
        link=card.find_all('a',href=True)[0]['href'].strip()
        title=card.find_all('h2',{'class':'home-title'})[0].text.strip().replace(u'\xa0', u' ')
        desc=card.find_all('div',{'class':'home-desc'})[0].text.strip().replace(u'\xa0', u' ')
        desc_text=str(desc)
        desc=desc_text.replace("\'","'")
        desc=desc+"..."
        '''
        tags=card.find_all('span',{'class':'h-tags'})
        if len(tags)>=1:
            tags=tags[0].text
        else:
            tags="Data Breach"
        b_items.append({'source':'TheHackerNews','link':link,'tags':tags,'title':title,'desc':desc})
        '''
        b_items.append({'source':'TheHackerNews','link':link,'title':title,'desc':desc})
    print(v_items)
    print()
    print(t_items)
    print()
    print(b_items)
    
    if v_items != None:
        vpath=filepath+"_vuln.json"
        json.dump(v_items,open(vpath,'w'))
    if t_items != None:
        tpath=filepath+"_trend.json"
        json.dump(t_items,open(tpath,'w'))
    if b_items != None:
        bpath=filepath+"_breach.json"
        json.dump(b_items,open(bpath,'w'))
    return v_items,t_items,b_items
v_path,t_path,b_path=thehackerNews()
#print(v_path,t_path,b_path)

A set of memory corruption flaws have been discovered in the  ncurses  (short for  new curses ) programming library that could be exploited by threat actors to run malicious code on vulnerable Linux and macOS systems. "Using environment variable poisoning, attackers could chain these vulnerabilities to elevate privileges and run code in the targeted program's context or perform other malicious actions," Microsoft Threat Intelligence researchers Jonathan Bar Or, Emanuele Cozzi, and Michael Pearse  said  in a technical report published today. The vulnerabilities, collectively tracked as  CVE-2023-29491  (CVSS score of 7.8), have been  addressed  as of April 2023. Microsoft said it also worked with Apple on addressing the macOS-specific issues related to these flaws. Environment variables are user-defined values that can be used by multiple programs on a system and can affect the manner in which they behave on the system. Manipulating the variables can cause application...
A high-severity

In [14]:
def dataBreaches(limit=5):
    filepath='logs/databreaches.json'
    baseURL = "https://www.databreaches.net/news/page/"
    items=[]
    for i in range(limit):
        URL = baseURL+str(i+1)+"/"
        req= Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
        webpage=urlopen(req).read()
        page_soup=BeautifulSoup(webpage,'html.parser')
        cards=page_soup.find_all('div',{'class':'entry-main'})
        for card in cards:
            title=card.find_all('h1',{'class':'entry-title'})[0].text.strip()
            link=card.find_all('a',href=True)[0]['href'].strip()
            desc=card.find_all('div',{'class':'entry-summary'})[0].text.replace(u'\xa0', u' ').strip()
            items.append({'source':'DataBreaches','title':title,'desc':desc,'link':link})
    #print(items)
    if items != None:
        json.dump(items,open(filepath,'w'))
        return items
    else: 
        return None
#path=dataBreaches(4)
#print(path)

In [15]:
def techCrunch():
    filepath='logs/techcrunch.json'
    URL = "https://techcrunch.com/category/security/"
    req= Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
    webpage=urlopen(req).read()
    page_soup=BeautifulSoup(webpage,'html.parser')
    cards=page_soup.find_all('a',{'class':'post-block__title__link'})
    items=[]
    for card in cards:
        title=card.text.strip()
        link=card['href'].strip()
        items.append({'source':'TechCrunch','title':title,'link':link})
    #print(items)
    if items != None:
        json.dump(items,open(filepath,'w'))
        return items
    else: 
        return None
#path=techCrunch()
#print(path)

In [18]:
def aggregate():
    headlines=[]
    cve_news=[]
    detailed_news=[]
    #---------------------------------------------------
    try:
        forti=fortinet()
    except:
        print('Fortinet Module is Crashing')
    else:
        if forti!=None:
            for item in forti:
                detailed_news.append(item)
    #----------------------------------------------------
    try:
        talos=talosintelligence()
    except:
        print('TalosIntelligence Module is Crashing')
    else:
        if talos!=None:
            for item in talos:
                cve_news.append(item)
    #---------------------------------------------------
    try:
        pp=proofPoint()
    except:
        print('Proofpoint Module is Crashing')
    else:
        if pp!=None:
            for item in pp:
                headlines.append(item)
    #---------------------------------------------------
    try:
        sw=securityWeek()
    except:
        print('SecurityWeek Module is Crashing')
    else:
        if sw!=None:
            for item in sw:
                headlines.append(item)
    #---------------------------------------------------
    try:
        s1=sentinelOne()
    except:
        print('SentinelOne Module is Crashing')
    else:
        if s1!=None:
            for item in s1:
                headlines.append(item)
    #---------------------------------------------------
    try:
        t=tenable()
    except:
        print('Tenable Module is Crashing')
    else:
        if t!=None:
            for item in t:
                headlines.append(item)
    #---------------------------------------------------
    try:
        rs=reSecurity()
    except:
        print('ReSecurity Module is Crashing')
    else:
        if rs!=None:
            for item in rs:
                headlines.append(item)
    #---------------------------------------------------
    try:
        tc=techCrunch()
    except:
        print('TechCrunch Module is Crashing')
    else:
        if tc!=None:
            for item in tc:
                headlines.append(item)
    print(headlines)
    print()
    print(cve_news)
    print()
    print(detailed_news)
    '''
    securityWeek()
    sentinelOne()
    
    try:
        forti=fortinet()
    except:
        print('Fortinet Module is Crashing')
    else:
        if forti!=None:
            for item in forti:
                detailed_news.append(item)

    try:
        forti=fortinet()
    except:
        print('Fortinet Module is Crashing')
    else:
        if forti!=None:
            for item in forti:
                detailed_news.append(item)
    try:
        forti=fortinet()
    except:
        print('Fortinet Module is Crashing')
    else:
        if forti!=None:
            for item in forti:
                detailed_news.append(item)
    try:
        forti=fortinet()
    except:
        print('Fortinet Module is Crashing')
    else:
        if forti!=None:
            for item in forti:
                detailed_news.append(item)
    try:
        forti=fortinet()
    except:
        print('Fortinet Module is Crashing')
    else:
        if forti!=None:
            for item in forti:
                detailed_news.append(item)
    '''
aggregate()

ReSecurity Module is Crashing
[{'source': 'ProofPoint', 'link': 'https://www.proofpoint.com/us/blog/threat-insight/out-sandbox-wikiloader-digs-sophisticated-evasion', 'title': 'Out of the Sandbox: WikiLoader Digs Sophisticated Evasion'}, {'source': 'ProofPoint', 'link': 'https://www.proofpoint.com/us/blog/threat-insight/job-scams-using-bioscience-lures-target-universities', 'title': 'Job Scams Using Bioscience Lures Target Universities'}, {'source': 'ProofPoint', 'link': 'https://www.proofpoint.com/us/blog/threat-insight/welcome-new-york-exploring-ta453s-foray-lnks-and-mac-malware', 'title': "Welcome to New York: Exploring TA453's Foray into LNKs and Mac Malware"}, {'source': 'ProofPoint', 'link': 'https://www.proofpoint.com/us/blog/email-and-cloud-threats/cloud-account-takeover-campaign-leveraging-evilproxy-targets-top-level', 'title': 'Cloud Account Takeover Campaign Leveraging EvilProxy Targets Top-Level Executives at over 100 Global Organizations'}, {'source': 'ProofPoint', 'link':

In [76]:
def recordedfuture():
    #filepath='logs/fortinet.json'
    keys=['technology','cybercrime','nation-state']
    items=[]
    for i in range(len(keys)):
        URL = "https://therecord.media/news/"+keys[i]+"/feed"
        req= Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
        webpage=urlopen(req).read()
        page_soup=BeautifulSoup(webpage,'xml')
        cards=page_soup.find_all('item')
        for card in cards:
            title=card.find_all('title')[0].text.strip()
            link=card.find_all('link')[0].text.strip()
            desc=str(card.find_all('description')[0].text.split('\n')[-1].strip()+"...")
            desc=desc.replace("&#39;","'")
            if "<a href=" in desc:
                desc=re.subn('<[a-z 0-9A-Z=":./\-%_]*>','',desc)[0]
            items.append({'title':title,'link':link,'desc':desc})
    URL = "https://therecord.media/"
    req= Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
    webpage=urlopen(req).read()
    page_soup=BeautifulSoup(webpage,'html.parser')
    #print(page_soup.prettify())
    # regex is (CyberVulnerability","entity_id":"[a-zA-Z0-9]*","curr_heat_score":[0-9]*,"name":")[a-zA-Z0-9\-",: .()']*,"link
    print(items)
recordedfuture()

[{'title': 'Lawsuit: ByteDance’s CapCut app secretly reaps massive amounts of user data', 'link': 'https://therecord.media/capcut-privacy-lawsuit-illinois-bipa-bytedance-china', 'desc': 'The ByteDance-owned CapCut video editing app gathers significant amounts of private data, including facial scans, from its 200 million active users, generating huge profits and potentially allowing the Chinese government to access that data, according to a proposed class action lawsuit filed in an Illinois federal court. CapCut and sister company TikTok are owned by...'}, {'title': 'FBI warns of adversaries using AI in influence campaigns, cyberattacks', 'link': 'https://therecord.media/fbi-artificial-intelligence-ai-briefing', 'desc': 'The FBI is paying increased attention to foreign adversaries’ attempts to utilize artificial intelligence as part of influence campaigns and other malicious activity, as well as their interest in tainting commercial AI software and stealing aspects of the emerging techn